In [23]:
#Section 2
#Question 9
import pandas as pd
import numpy as np

def calculate_distance_matrix(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate a distance matrix based on the dataframe, df.

    Args:
        df (pandas.DataFrame): DataFrame containing 'id_start', 'id_end', and 'distance'.

    Returns:
        pandas.DataFrame: Distance matrix
    """
    # Check if required columns are in the DataFrame
    required_columns = ['id_start', 'id_end', 'distance']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' is not in the DataFrame. Available columns: {df.columns.tolist()}")

    # Create a set of unique IDs
    unique_ids = pd.unique(df[['id_start', 'id_end']].values.ravel('K'))

    # Initialize the distance matrix with infinities
    distance_matrix = pd.DataFrame(np.inf, index=unique_ids, columns=unique_ids)

    # Populate the distance matrix with known distances
    for _, row in df.iterrows():
        distance_matrix.at[row['id_start'], row['id_end']] = row['distance']
        distance_matrix.at[row['id_end'], row['id_start']] = row['distance']  # Ensure symmetry

    # Set diagonal values to 0
    np.fill_diagonal(distance_matrix.values, 0)

    # Use the Floyd-Warshall algorithm to calculate cumulative distances
    for k in unique_ids:
        for i in unique_ids:
            for j in unique_ids:
                if distance_matrix.at[i, k] + distance_matrix.at[k, j] < distance_matrix.at[i, j]:
                    distance_matrix.at[i, j] = distance_matrix.at[i, k] + distance_matrix.at[k, j]

    return distance_matrix

# Example usage
if __name__ == "__main__":
    # Load the dataset
    df = pd.read_csv('dataset-2.csv')

    # Calculate the distance matrix
    distance_matrix = calculate_distance_matrix(df)
    print("Distance Matrix:")
    print(distance_matrix)

#Question 10
import pandas as pd

def unroll_distance_matrix(distance_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Unroll a distance matrix to a DataFrame in the style of the initial dataset.

    Args:
        distance_matrix (pandas.DataFrame): The distance matrix with IDs as both index and columns.

    Returns:
        pandas.DataFrame: Unrolled DataFrame containing columns 'id_start', 'id_end', and 'distance'.
    """
    # Create a list to hold the rows for the unrolled DataFrame
    unrolled_data = []

    # Iterate over the DataFrame index (id_start)
    for id_start in distance_matrix.index:
        # Iterate over the DataFrame columns (id_end)
        for id_end in distance_matrix.columns:
            # Skip the entry if id_start is the same as id_end (diagonal entries)
            if id_start != id_end:
                distance = distance_matrix.loc[id_start, id_end]
                unrolled_data.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})

    # Convert the list of dictionaries to a DataFrame
    unrolled_df = pd.DataFrame(unrolled_data)

    return unrolled_df

# Example usage
# distance_matrix is the DataFrame created in the previous step
unrolled_df = unroll_distance_matrix(distance_matrix)
print(unrolled_df) 


# Question 11
import pandas as pd

def find_ids_within_ten_percentage_threshold(df: pd.DataFrame, reference_id: int) -> pd.DataFrame:
    """
    Find all IDs whose average distance lies within 10% of the average distance of the reference ID.

    Args:
        df (pandas.DataFrame): DataFrame with 'id_start', 'id_end', and 'distance'.
        reference_id (int): The reference ID to compare against.

    Returns:
        pandas.DataFrame: DataFrame with IDs whose average distance is within the specified percentage threshold
                          of the reference ID's average distance.
    """
    # Calculate average distance for the reference_id
    reference_avg = df[df['id_start'] == reference_id]['distance'].mean()
    
    # Calculate the 10% threshold
    lower_bound = reference_avg * 0.9
    upper_bound = reference_avg * 1.1

    # Group by id_start and calculate the average distance for each ID
    avg_distances = df.groupby('id_start')['distance'].mean().reset_index()

    # Filter IDs within the 10% threshold
    filtered_ids = avg_distances[
        (avg_distances['distance'] >= lower_bound) & 
        (avg_distances['distance'] <= upper_bound)
    ]

    # Sort the filtered IDs by distance
    sorted_filtered_ids = filtered_ids.sort_values(by='distance')

    return sorted_filtered_ids

# Example usage:
# unrolled_df is the DataFrame from the previous step
reference_id = 1  # Example reference ID
result_df = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
print(result_df)

#Question 12
import pandas as pd

def calculate_toll_rate(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate toll rates for each vehicle type based on the unrolled DataFrame.

    Args:
        df (pandas.DataFrame): Input DataFrame with 'distance' column.

    Returns:
        pandas.DataFrame: DataFrame with additional columns for each vehicle type's toll rate.
    """
    # Define the rate coefficients for each vehicle type
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }
    
    # Calculate toll rates and add new columns
    for vehicle, coefficient in rate_coefficients.items():
        df[vehicle] = df['distance'] * coefficient
    
    return df

# Sample usage:
# Assuming `unrolled_df` is the DataFrame from Question 10
result_df = calculate_toll_rate(unrolled_df)
print(result_df)


#Question 13
import pandas as pd
from datetime import time

def calculate_time_based_toll_rates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate time-based toll rates for different time intervals within a day.

    Args:
        df (pandas.DataFrame): Input DataFrame with distance and toll rates.

    Returns:
        pandas.DataFrame: DataFrame with time-based toll rates in the specified format.
    """
    new_rows = []

    # Define time ranges and discount factors
    weekday_discount = { 
        (time(0, 0), time(10, 0)): 0.8,
        (time(10, 0), time(18, 0)): 1.2,
        (time(18, 0), time(23, 59, 59)): 0.8
    }
    
    weekend_discount = 0.7

    # Iterate through each unique (id_start, id_end) pair
    for _, group in df.groupby(['id_start', 'id_end']):
        distance = group['distance'].values[0]  # Get distance for the current pair

        for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
            is_weekend = day in ['Saturday', 'Sunday']

            for hour in range(24):
                for minute in [0, 30]:  # Half-hour increments
                    start_time = time(hour, minute)
                    end_time = time((hour + 1) % 24, minute)  # End time is one hour later

                    # Calculate toll rates for each vehicle
                    toll_rates = {}
                    for vehicle in ['moto', 'car', 'rv', 'bus', 'truck']:
                        if vehicle in group.columns:
                            base_rate = group[vehicle].values[0]  # Base rate from the group
                        else:
                            base_rate = 0

                        # Determine the discount factor
                        if is_weekend:
                            discount_factor = weekend_discount
                        else:
                            discount_factor = 1.0  # Default
                            for time_range, factor in weekday_discount.items():
                                if time_range[0] <= start_time < time_range[1]:
                                    discount_factor = factor
                                    break

                        # Calculate the toll rate
                        toll_rate = base_rate * discount_factor
                        toll_rates[vehicle] = toll_rate

                    # Append the new row with the correct column order
                    new_rows.append({
                        'id_start': group['id_start'].values[0],
                        'id_end': group['id_end'].values[0],
                        'distance': distance,
                        'start_day': day,
                        'start_time': start_time,
                        'end_time': end_time,
                        'moto': toll_rates['moto'],
                        'car': toll_rates['car'],
                        'rv': toll_rates['rv'],
                        'bus': toll_rates['bus'],
                        'truck': toll_rates['truck']
                    })

    # Create a new DataFrame from the new rows
    result_df = pd.DataFrame(new_rows)

    # Ensure the output has the correct column order
    result_df = result_df[['id_start', 'id_end', 'distance', 'start_day', 'start_time', 'end_time', 'moto', 'car', 'rv', 'bus', 'truck']]
    
    return result_df

# Call the function with your DataFrame
result_df = calculate_time_based_toll_rates(toll_df)

# Print the resulting DataFrame
print(result_df)


Distance Matrix:
         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   
1001410     78.7     69.0     48.8     32.8     11.1      0.0     15.6   
1001412     94.3     84.6     64.4     48.4     26.7     15.6      0.0   
1001414    112.5    102.8     82.6     66.6     44.9     33.8     18.2   
1001416    125.7    116.0     95.8     79.8     58.1     47.0     31.4   
1001418    139.3    129.6    109.4     93.4     71.7     60.6     45.0   
1001420    152.2    142.5    122.3    106.3     84.6     73.5     57.9   
1001422    161.8    152.1    131.9    115.9     94.2     83.1     67.5   
1001424    173.2    1